# Fine Tuning BART

<a href="https://colab.research.google.com/github/hjesse92/style_transfer_w266/blob/main/notebooks/BART_Models.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Setup

In [1]:
!pip install -q transformers datasets rouge_score accelerate evaluate

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.8/6.8 MB 44.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 468.7/468.7 KB 32.0 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 215.3/215.3 KB 18.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 81.4/81.4 KB 7.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.6/7.6 MB 54.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 199.8/199.8 KB 25.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 212.2/212.2 KB 28.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 132.9/132.9 KB 9.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 110.5/110.5 KB 15.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 40.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 114.2/114.2 KB 15.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 158.8/158.8 KB 22.7 MB/s eta 0:

In [2]:
#Am I running a GPU and what type is it?
!nvidia-smi

Sat Apr  1 22:57:36 2023       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 525.85.12    Driver Version: 525.85.12    CUDA Version: 12.0     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   55C    P8    10W /  70W |      0MiB / 15360MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [3]:
import torch

# Clear out cuda
torch.cuda.empty_cache()

if torch.cuda.is_available():     
    device = torch.device("cuda")
    print('Number of GPU(s) available:', torch.cuda.device_count())
    print('GPU device name:', torch.cuda.get_device_name(0))

else:
    print('No GPU available')
    device = torch.device("cpu")

Number of GPU(s) available: 1
GPU device name: Tesla T4


In [4]:
from logging import warning
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader, RandomSampler, SequentialSampler, TensorDataset

from transformers import AutoModelForSeq2SeqLM, DataCollatorForSeq2Seq, AdamW, Seq2SeqTrainingArguments, Seq2SeqTrainer
from transformers import AutoTokenizer, BartForConditionalGeneration
from datasets import load_metric, load_dataset


from sklearn.utils import resample
from sklearn.model_selection import train_test_split

import re
import random
import pandas as pd
import seaborn as sns
import numpy as np
import matplotlib.pyplot as plt
import pprint
import nltk

import warnings
warnings.filterwarnings('ignore')

RANDOM_SEED = 42
np.random.seed(RANDOM_SEED)
torch.manual_seed(RANDOM_SEED)
nltk.download('punkt')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

In [5]:
from google.colab import drive
drive.mount('/content/drive')


Mounted at /content/drive


In [6]:
cd drive/MyDrive/w266/style_transfer_w266/

/content/drive/MyDrive/w266/style_transfer_w266


In [ ]:
train_file = 'data/original-train.tsv'
dev_file = 'data/original-dev.tsv'
test_file = 'data/original-test.tsv'
df_train = pd.read_csv(train_file, sep='\t')
df_dev = pd.read_csv(dev_file, sep='\t')
df_test = pd.read_csv(test_file, sep='\t')

In [ ]:
print(f'''mean length of offensive text: {df_train['offensive-text'].map(len).mean()}''')
print(f'''min length of offensive text: {df_train['offensive-text'].map(len).min()}''')
print(f'''max length of offensive text: {df_train['offensive-text'].map(len).max()}''')
print(f'''mean length of neutralized text: {df_train['style-transferred-text'].map(len).mean()}''')
print(f'''min length of neutralized text: {df_train['style-transferred-text'].map(len).min()}''')
print(f'''max length of neutralized text: {df_train['style-transferred-text'].map(len).max()}''')

mean length of offensive text: 69.85353535353535
min length of offensive text: 9
max length of offensive text: 238
mean length of neutralized text: 60.48800505050505
min length of neutralized text: 1
max length of neutralized text: 174


## Train BART with Trainer

In [ ]:
dataset = load_dataset('csv', sep="\t", data_files={'train': train_file, 'validation': dev_file,'test': test_file})

Extracting data files:   0%|          | 0/3 [00:00<?, ?it/s]

Generating train split: 0 examples [00:00, ? examples/s]

Generating validation split: 0 examples [00:00, ? examples/s]

Generating test split: 0 examples [00:00, ? examples/s]

Dataset csv downloaded and prepared to /root/.cache/huggingface/datasets/csv/default-1fb4d7f9839c26ad/0.0.0/6b34fb8fcf56f7c8ba51dc895bfa2bfbe43546f190a60fcf74bb5e8afdcc2317. Subsequent calls will reuse this data.


  0%|          | 0/3 [00:00<?, ?it/s]

In [ ]:
tokenizer = AutoTokenizer.from_pretrained("facebook/bart-base")

In [ ]:
prefix = 'transfer to nontoxic text: '
max_input_length = 64
max_target_length = 64


def preprocess_data(examples):
  source_inputs = [prefix + text for text in examples['offensive-text']]
  model_inputs = tokenizer(source_inputs, max_length=max_input_length, padding="max_length", truncation=True)
  
  target_inputs = [text for text in examples['style-transferred-text']]
  target_tokens = tokenizer(target_inputs, max_length=max_target_length, padding="max_length", truncation=True)
  
  # important: we need to replace the index of the padding tokens by -100
  # such that they are not taken into account by the CrossEntropyLoss
  labels_with_ignore_index = []
  for labels_example in target_tokens.input_ids:
    labels_example = [label if label != 0 else -100 for label in labels_example]
    labels_with_ignore_index.append(labels_example)
  
  model_inputs["labels"] = labels_with_ignore_index
  # model_inputs["labels"] = target_tokens
  
  return model_inputs

In [ ]:
tokenized_datasets = dataset.map(preprocess_data, batched=True)
encoded_train_ds = dataset['train'].map(preprocess_data, batched=True, remove_columns=dataset['train'].column_names)
encoded_val_ds = dataset['validation'].map(preprocess_data, batched=True, remove_columns=dataset['validation'].column_names)
encoded_test_ds = dataset['test'].map(preprocess_data, batched=True, remove_columns=dataset['test'].column_names)

Map:   0%|          | 0/1584 [00:00<?, ? examples/s]

Map:   0%|          | 0/198 [00:00<?, ? examples/s]

Map:   0%|          | 0/199 [00:00<?, ? examples/s]

Map:   0%|          | 0/1584 [00:00<?, ? examples/s]

Map:   0%|          | 0/198 [00:00<?, ? examples/s]

Map:   0%|          | 0/199 [00:00<?, ? examples/s]

In [ ]:
# preprocess_data(dataset['train'][0])
# tokenized_datasets['train']['input_ids'][0][0]
tokenizer.decode(tokenized_datasets['train'][0]['input_ids'])

'<s>transfer to nontoxic text: Pussy nobody asked for your input.</s><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad>'

In [ ]:
tokenizer.decode(encoded_train_ds[0]['input_ids'])

'<s>transfer to nontoxic text: Pussy nobody asked for your input.</s><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad>'

In [ ]:
tokenized_datasets

DatasetDict({
    train: Dataset({
        features: ['offensive-text', 'style-transferred-text', 'input_ids', 'attention_mask', 'labels'],
        num_rows: 1584
    })
    validation: Dataset({
        features: ['offensive-text', 'style-transferred-text', 'input_ids', 'attention_mask', 'labels'],
        num_rows: 198
    })
    test: Dataset({
        features: ['offensive-text', 'style-transferred-text', 'input_ids', 'attention_mask', 'labels'],
        num_rows: 199
    })
})

In [ ]:
encoded_train_ds

Dataset({
    features: ['input_ids', 'attention_mask', 'labels'],
    num_rows: 1584
})

In [ ]:
batch_size = 8
model_name = "facebook/bart-base-detoxify"
model_dir = f"models/{model_name}"

args = Seq2SeqTrainingArguments(
    model_dir,
    evaluation_strategy="epoch",
    logging_strategy="epoch",
    save_strategy="epoch",
    learning_rate=2e-5,
    per_device_train_batch_size=batch_size,
    per_device_eval_batch_size=batch_size,
    weight_decay=0.01,
    num_train_epochs=7,
    predict_with_generate=True,
    load_best_model_at_end=True,
    metric_for_best_model="rouge1",
    remove_unused_columns=False
)

In [ ]:
data_collator = DataCollatorForSeq2Seq(tokenizer)
metric = load_metric("rouge")

In [ ]:
import nltk

def compute_metrics(eval_pred):
    predictions, labels = eval_pred
    decoded_preds = tokenizer.batch_decode(predictions, skip_special_tokens=True)
    
    # Replace -100 in the labels as we can't decode them.
    labels = np.where(labels != -100, labels, tokenizer.pad_token_id)
    decoded_labels = tokenizer.batch_decode(labels, skip_special_tokens=True)
    
    # Rouge expects a newline after each sentence
    decoded_preds = ["\n".join(nltk.sent_tokenize(pred.strip()))
                      for pred in decoded_preds]
    decoded_labels = ["\n".join(nltk.sent_tokenize(label.strip())) 
                      for label in decoded_labels]
    
    # Compute ROUGE scores
    result = metric.compute(predictions=decoded_preds, references=decoded_labels,
                            use_stemmer=True)

    # Extract ROUGE f1 scores
    result = {key: value.mid.fmeasure * 100 for key, value in result.items()}
    
    # Add mean generated length to metrics
    prediction_lens = [np.count_nonzero(pred != tokenizer.pad_token_id)
                      for pred in predictions]
    result["gen_len"] = np.mean(prediction_lens)
    
    return {k: round(v, 4) for k, v in result.items()}

In [ ]:
from transformers import AdamW, get_cosine_schedule_with_warmup

model_ckpt = "facebook/bart-base"
model = BartForConditionalGeneration.from_pretrained(model_ckpt)
model = model.to(device)

optimizer = AdamW(model.parameters(), lr=2e-5)
scheduler = get_cosine_schedule_with_warmup(optimizer, num_warmup_steps=10, num_training_steps=1000)
# scheduler = torch.optim.lr_scheduler.StepLR(optimizer, 1.0, gamma=0.95)

In [ ]:
trainer = Seq2SeqTrainer(
    model=model,
    args=args,
    # train_dataset=tokenized_datasets['train'],
    train_dataset=encoded_train_ds, 
    # eval_dataset=tokenized_datasets['validation'],
    eval_dataset=encoded_val_ds,
    data_collator=data_collator,
    tokenizer=tokenizer,
    optimizers=(optimizer, scheduler),
    compute_metrics=compute_metrics
)

In [ ]:
# %load_ext tensorboard
# %tensorboard --logdir '{model_dir}'/runs

In [ ]:
trainer.train()

Epoch,Training Loss,Validation Loss,Rouge1,Rouge2,Rougel,Rougelsum,Gen Len
1,3.897100,0.476778,66.728200,53.412300,66.506400,66.531600,16.878800
2,0.354200,0.272928,68.972500,56.506700,68.929300,68.855100,16.247500
3,0.234900,0.265647,69.151500,56.607000,69.152400,69.158300,16.368700
4,0.201300,0.264540,69.328900,56.788400,69.287700,69.325400,16.383800
5,0.186900,0.266082,69.842500,57.423700,69.815800,69.769700,16.232300
6,0.185900,0.265690,69.595300,57.122100,69.587700,69.621900,16.358600
7,0.186000,0.266540,69.420300,56.980500,69.423300,69.381800,16.434300


TrainOutput(global_step=1386, training_loss=0.7494761892211386, metrics={'train_runtime': 419.2494, 'train_samples_per_second': 26.447, 'train_steps_per_second': 3.306, 'total_flos': 422547365560320.0, 'train_loss': 0.7494761892211386, 'epoch': 7.0})

In [ ]:
# save training weights
trainer.save_model('models/bart-detoxify-base')
torch.save(model.state_dict(), 'models/bart-detoxify-base1.pth')

In [ ]:
# Calculate rouge score for test set
trainer.evaluate(encoded_test_ds)

{'eval_loss': 0.26057571172714233,
 'eval_rouge1': 69.671,
 'eval_rouge2': 57.823,
 'eval_rougeL': 69.1977,
 'eval_rougeLsum': 69.2799,
 'eval_gen_len': 15.7688,
 'eval_runtime': 11.3529,
 'eval_samples_per_second': 17.529,
 'eval_steps_per_second': 2.202,
 'epoch': 7.0}

In [ ]:
model.eval()

test_dataloader = DataLoader(encoded_test_ds, batch_size=8, collate_fn=data_collator)

generated_texts = []
with torch.no_grad():
    for batch in test_dataloader:
        input_ids = batch['input_ids'].to(device)
        attention_mask = batch['attention_mask'].to(device)
        output = model.generate(input_ids=input_ids, attention_mask=attention_mask, max_length=64)
        generated_texts.extend(tokenizer.batch_decode(output, skip_special_tokens=True))



In [ ]:
generated_texts[:10]

['So maybe you should be more mature.',
 "THERES A MEGATHREAD FOR VACCINE OR COVID RELATED TOPICS. DON'T TALK ABOUT THAT.",
 'you are on drugs',
 'NTA.   Dump his tablemanners. Treating other people like human beings is no rocket science. His still a child.',
 'Youre soft as baby',
 "Because everyone knows why. It's jadas fault. Will was just triggered because of all the stuff he has to deal with because of her.",
 'And “poor kids are just as bright as white kids”…. Your ignorance chooses racism as well',
 "Tucker is a bad person that no one will miss when he's gone… he’s also a racist",
 "There's plenty of evidence, it's exactly why they won't allow forensic audits   *wipes voting machines clean of their records*",
 ' Came here to say this, carlsen is not very smart and defines recreational outrage imo']

In [ ]:
# Create a new DataFrame with the generated texts
df_gen = pd.DataFrame({'bart_tuning_output': generated_texts})

# Add the generated_text column to the df_test dataframe
df_output = pd.concat([df_test, df_gen], axis=1)

df_output.head(10)

,offensive-text,style-transferred-text,bart_tuning_output
0,So maybe you should be more retarded.,So maybe you should be more backward,So maybe you should be more mature.
1,THERES A MEGATHREAD FOR VACCINE OR COVID RELAT...,THERES ACTUALLY A MEGATHREAD FOR VACCINE OR CO...,THERES A MEGATHREAD FOR VACCINE OR COVID RELAT...
2,the fuck.... you are on drugs,uh..... you are not being realistic,you are on drugs
3,NTA. Dump his ass. Tablemanners are no rocke...,You should leave him. Tablemanners are no rock...,NTA. Dump his tablemanners. Treating other p...
4,Youre soft as baby shit,Youre really soft,Youre soft as baby
5,Because everyone knows why. It’s jadas fault. ...,Because everyone knows it's jadas fault. Will ...,Because everyone knows why. It's jadas fault. ...
6,And “poor kids are just as bright as white kid...,"He said ""poor kids are just as bright as white...",And “poor kids are just as bright as white kid...
7,Tucker is a garbage person that no one will mi...,Tucker is a bad person who instigates racism,Tucker is a bad person that no one will miss w...
8,"There’s plenty of evidence, it’s exactly why t...","There is evidence, which is why they won't all...","There's plenty of evidence, it's exactly why t..."
9,"Came here to say this, carlsen is a freaking m...","Came here to say this, carlsen is pretty unint...","Came here to say this, carlsen is not very sm..."


In [ ]:
# Save the updated dataframe as a CSV file
df_output.to_csv('outputs/bart_tuning_output.csv',sep='\t',index=False)

In [7]:
import sys
sys.path.append('./notebooks')
from DistilBertClassification import BertClassificationML, NonToxicScoreDataLoader, NonToxicScore

# Load DistilBERT Classification Model to calculate NonToxicScore
score_model = BertClassificationML()
score_model = score_model.to(device)

# Load training weights
pretrained_weights = torch.load('models/DistilBertToxicClassification7.pth')
score_model.load_state_dict(pretrained_weights )

Some weights of the model checkpoint at distilbert-base-uncased were not used when initializing DistilBertModel: ['vocab_layer_norm.bias', 'vocab_transform.weight', 'vocab_transform.bias', 'vocab_projector.weight', 'vocab_projector.bias', 'vocab_layer_norm.weight']
- This IS expected if you are initializing DistilBertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing DistilBertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


<All keys matched successfully>

In [11]:
## Calculate NonToxicScore
output_file = 'outputs/bart_tuning_output.csv'
output_col = 'bart_tuning_output'

# Create Data Loader
score_loader = NonToxicScoreDataLoader(output_file, output_col, max_length=512)

# Calculate NonToxicScore
bart_NonToxicScores, avg_score = NonToxicScore(score_loader, score_model)

{'NonToxicScore': 0.5762933659584883}


In [13]:
# Look at output texts with low NonToxicScore, or still classified as Toxic after Style Transfering
df_finetune = pd.read_csv('outputs/bart_tuning_output.csv',sep='\t')
df_finetune['NonToxicScore'] = bart_NonToxicScores

df_finetune_lowscore = df_finetune[df_finetune['NonToxicScore'] < 0.5][['offensive-text', 'style-transferred-text', 'bart_tuning_output', 'NonToxicScore']]
df_finetune_lowscore

,offensive-text,style-transferred-text,bart_tuning_output,NonToxicScore
3,NTA. Dump his ass. Tablemanners are no rocke...,You should leave him. Tablemanners are no rock...,NTA. Dump his tablemanners. Treating other p...,0.025178
6,And “poor kids are just as bright as white kid...,"He said ""poor kids are just as bright as white...",And “poor kids are just as bright as white kid...,0.020789
7,Tucker is a garbage person that no one will mi...,Tucker is a bad person who instigates racism,Tucker is a bad person that no one will miss w...,0.111267
9,"Came here to say this, carlsen is a freaking m...","Came here to say this, carlsen is pretty unint...","Came here to say this, carlsen is not very sm...",0.485712
10,Fuck off dude. Worst of the fucking worst!,Stop dude. Worst of the worst!,Leave dude. Worst of the worst!,0.251839
...,...,...,...,...
187,Anti gay equals anti human. Fuck you Putin nazis,Anti gay equals anti human,Anti gay equals anti human.,0.006351
190,You’ve been completely subverted yet here you ...,It's a little ironic that you've been subverte...,You've been completely subverted yet here you ...,0.200510
191,I wish I could spit my unvaccinated saliva on ...,I wish I could weaponize my unvaccinated saliva.,I wish I could spit on you and your family.,0.013394
195,Formula one…nascar….shit it exactly the same e...,Formula one…nascar….it exactly the same except...,Formula one…nascar….shit it exactly the same e...,0.005856


In [17]:
for i in range(len(df_finetune_lowscore)):
    print('Toxic Text:', df_finetune_lowscore['offensive-text'].values[i])
    print('Style Transfer Text:', df_finetune_lowscore.bart_tuning_output.values[i])
    print('NonToxicScore:', df_finetune_lowscore.NonToxicScore.values[i])

Toxic Text: NTA.   Dump his ass. Tablemanners are no rocket science. Treating other people like human beings is no rocket science. His still a child.
Style Transfer Text: NTA.   Dump his tablemanners. Treating other people like human beings is no rocket science. His still a child.
NonToxicScore: 0.025178326293826103
Toxic Text: And “poor kids are just as bright as white kids”…. Your blundering bafoon chooses racism as well
Style Transfer Text: And “poor kids are just as bright as white kids”…. Your ignorance chooses racism as well
NonToxicScore: 0.020789258182048798
Toxic Text: Tucker is a garbage person that no one will miss when he’s gone… he’s also a racist ruble-rouser
Style Transfer Text: Tucker is a bad person that no one will miss when he's gone… he’s also a racist
NonToxicScore: 0.11126653105020523
Toxic Text: Came here to say this, carlsen is a freaking moron and defines recreational outrage imo
Style Transfer Text:  Came here to say this, carlsen is not very smart and defines